In [45]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json


In [16]:
hex_proof = gpd.read_file('./unsynced-data/rwanda/rwa_travel_time_hex-8-staging.geojson')
hex_data = hex_proof.copy()
hex_data

,h3-index,row_col,female_educational_attainment_mean,females_0_4,females_0_9,females_10_14,females_15_49,females_50_64,females_5_9,females_65_plus,...,travel_time_no_sites_health_centers_optimal,travel_time_no_sites_health_posts_optimal,travel_time_no_sites_major_hospitals_optimal,travel_time_no_sites_primary_schools_fixed,travel_time_no_sites_secondary_schools_fixed,travel_time_no_sites_semi_dense_urban_optimal,travel_time_primary_schools_fixed,travel_time_secondary_schools_fixed,travel_time_semi_dense_urban_optimal,geometry
0,886ad80001fffff,1334_1576&1335_1573&1335_1574&1335_1575&1335_1...,4.5768,1,2,1,5,0,1,0,...,200.0,113.0,333.0,67.0,162.0,185.0,67.0,162.0,185.0,"POLYGON ((30.17441 -2.16518, 30.17633 -2.16048..."
1,886ad80003fffff,1338_1586&1338_1587&1338_1588&1339_1583&1339_1...,4.5731,0,0,0,1,0,0,0,...,181.0,105.0,341.0,61.0,132.0,182.0,61.0,132.0,182.0,"POLYGON ((30.18272 -2.16847, 30.18463 -2.16378..."
2,886ad80005fffff,1343_1564&1344_1563&1344_1564&1344_1572&1344_1...,4.6131,4,10,7,24,4,6,1,...,130.0,68.0,286.0,43.0,172.0,120.0,43.0,172.0,120.0,"POLYGON ((30.16737 -2.17058, 30.16929 -2.16588..."
3,886ad80007fffff,1344_1575&1344_1577&1345_1578&1345_1579&1346_1...,4.5900,1,3,2,8,1,2,0,...,169.0,104.0,369.0,38.0,132.0,157.0,38.0,132.0,157.0,"POLYGON ((30.17568 -2.17387, 30.17760 -2.16918..."
4,886ad80009fffff,1323_1573&1324_1572&1324_1573&1325_1573&1325_1...,4.5728,1,2,1,6,0,1,0,...,205.0,82.0,280.0,64.0,185.0,172.0,64.0,185.0,172.0,"POLYGON ((30.17313 -2.15649, 30.17505 -2.15179..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26179,886adeb765fffff,760_604&761_599&761_602&761_603&761_604&762_60...,4.2135,69,147,76,279,37,77,14,...,46.0,45.0,344.0,31.0,31.0,80.0,31.0,31.0,80.0,"POLYGON ((29.36233 -1.68778, 29.36426 -1.68310..."
26180,886adeb767fffff,770_606&770_607&770_608&771_609&771_610&772_60...,NaN,0,0,0,0,0,0,0,...,71.0,68.0,367.0,47.0,53.0,89.0,47.0,53.0,89.0,"POLYGON ((29.37061 -1.69107, 29.37255 -1.68639..."
26181,886adeb769fffff,744_606&744_610&744_611&745_606&745_611&745_61...,4.1503,37,79,41,151,20,42,7,...,30.0,78.0,373.0,44.0,44.0,95.0,43.0,43.0,95.0,"POLYGON ((29.36812 -1.67374, 29.37006 -1.66906..."
26182,886adeb76dfffff,750_601&750_602&750_603&751_602&751_603&752_60...,4.2824,126,267,139,508,67,141,25,...,15.0,46.0,340.0,31.0,31.0,95.0,30.0,30.0,94.0,"POLYGON ((29.36109 -1.67912, 29.36302 -1.67444..."


In [14]:
hex_data['travel_time_no_sites_all_education_facilities_fixed']

0        67.04000000000001
1        60.94871794871795
2        42.57142857142857
3        38.39130434782609
4        64.18918918918919
               ...        
26179    30.63793103448276
26180    46.77777777777778
26181    44.26530612244898
26182    31.15714285714286
26183                   27
Name: travel_time_no_sites_all_education_facilities_fixed, Length: 26184, dtype: object

In [10]:
school_bins = [30, 45, 60]
health_bins = [45, 90, 135]
hospital_bins = [60, 120, 180]
market_bins = [60, 120, 180]

In [39]:
scoring_rubric =[[0, 0, 0, 0], [2, 0, 0, 0], [4, 2, 0, 0], [8, 4, 2, 0]]

def calculate_impact_score(before_travel_time, after_travel_time, bins):
    # Calculate the bin indices for before and after travel times
    before_bin = np.digitize(before_travel_time, bins)
    after_bin = np.digitize(after_travel_time, bins)
    
    # Initialize an array to store impact scores
    impact_scores = np.zeros(len(before_travel_time))
    
    # Loop through each row and calculate the impact score
    for i in range(len(before_travel_time)):
        impact_scores[i] = scoring_rubric[before_bin[i]][after_bin[i]]
        if impact_scores[i] == 0 and before_travel_time[i] != after_travel_time[i] and not np.isnan(after_travel_time[i]):
            print(before_travel_time[i], after_travel_time[i], before_bin[i], after_bin[i])
            impact_scores[i] = 1

    return impact_scores

In [41]:
test = calculate_impact_score(hex_data['travel_time_no_sites_all_education_facilities_fixed'], hex_data['travel_time_all_education_facilities_fixed'], school_bins)
test


82.0 83.0 3 3
91.0 78.0 3 3
126.0 67.0 3 3
36.0 30.0 1 1
69.0 60.0 3 3
360.0 61.0 3 3
nan 155.0 3 3
470.0 80.0 3 3
nan 126.0 3 3
nan 111.0 3 3
nan 143.0 3 3
104.0 103.0 3 3
78.0 93.0 3 3
194.0 115.0 3 3
448.0 102.0 3 3
nan 159.0 3 3
nan 181.0 3 3
nan 137.0 3 3
nan 165.0 3 3
159.0 160.0 3 3
nan 199.0 3 3
173.0 178.0 3 3
229.0 223.0 3 3
122.0 71.0 3 3
173.0 82.0 3 3
50.0 49.0 2 2
56.0 55.0 2 2
53.0 47.0 2 2
23.0 15.0 0 0
50.0 46.0 2 2
19.0 17.0 0 0
54.0 52.0 2 2
25.0 22.0 0 0
62.0 61.0 3 3
84.0 74.0 3 3
204.0 199.0 3 3
42.0 41.0 1 1
54.0 50.0 2 2
43.0 32.0 1 1
28.0 27.0 0 0
38.0 37.0 1 1
94.0 69.0 3 3
398.0 64.0 3 3
425.0 73.0 3 3
366.0 77.0 3 3
258.0 112.0 3 3
405.0 77.0 3 3
167.0 90.0 3 3
403.0 102.0 3 3
237.0 105.0 3 3
406.0 97.0 3 3
41.0 34.0 1 1
89.0 88.0 3 3
13.0 12.0 0 0
17.0 15.0 0 0
20.0 19.0 0 0
22.0 21.0 0 0
25.0 24.0 0 0
44.0 39.0 1 1
9.0 8.0 0 0
180.0 68.0 3 3
259.0 64.0 3 3
103.0 81.0 3 3
183.0 91.0 3 3
237.0 110.0 3 3
208.0 85.0 3 3
241.0 87.0 3 3
44.0 43.0 1 1
55.0 45.0 2

array([0., 0., 0., ..., 1., 1., 0.])

In [42]:
hex_data['all_education_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_all_education_facilities_fixed'], hex_data['travel_time_all_education_facilities_fixed'], school_bins)
hex_data['all_education_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_all_education_facilities_fixed'], hex_data['travel_time_constructed_sites_all_education_facilities_fixed'], school_bins)
hex_data['all_health_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_all_health_facilities_optimal'], hex_data['travel_time_all_health_facilities_optimal'], health_bins)
hex_data['all_health_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_all_health_facilities_optimal'], hex_data['travel_time_constructed_sites_all_health_facilities_optimal'], health_bins)
hex_data['health_centers_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_health_centers_optimal'], hex_data['travel_time_health_centers_optimal'], health_bins)
hex_data['health_centers_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_health_centers_optimal'], hex_data['travel_time_constructed_sites_health_centers_optimal'], health_bins)
hex_data['health_posts_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_health_posts_optimal'], hex_data['travel_time_health_posts_optimal'], health_bins)
hex_data['health_posts_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_health_posts_optimal'], hex_data['travel_time_constructed_sites_health_posts_optimal'], health_bins)
hex_data['major_hospitals_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_major_hospitals_optimal'], hex_data['travel_time_major_hospitals_optimal'], hospital_bins)
hex_data['major_hospitals_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_major_hospitals_optimal'], hex_data['travel_time_constructed_sites_major_hospitals_optimal'], hospital_bins)
hex_data['primary_schools_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_primary_schools_fixed'], hex_data['travel_time_primary_schools_fixed'], school_bins)
hex_data['primary_schools_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_primary_schools_fixed'], hex_data['travel_time_constructed_sites_primary_schools_fixed'], school_bins)
hex_data['secondary_schools_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_secondary_schools_fixed'], hex_data['travel_time_secondary_schools_fixed'], school_bins)
hex_data['secondary_schools_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_secondary_schools_fixed'], hex_data['travel_time_constructed_sites_secondary_schools_fixed'], school_bins)
hex_data['semi_dense_urban_potential_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_semi_dense_urban_optimal'], hex_data['travel_time_semi_dense_urban_optimal'], market_bins)
hex_data['semi_dense_urban_current_impact'] = calculate_impact_score(hex_data['travel_time_no_sites_semi_dense_urban_optimal'], hex_data['travel_time_constructed_sites_semi_dense_urban_optimal'], market_bins)

82.0 83.0 3 3
91.0 78.0 3 3
126.0 67.0 3 3
36.0 30.0 1 1
69.0 60.0 3 3
360.0 61.0 3 3
nan 155.0 3 3
470.0 80.0 3 3
nan 126.0 3 3
nan 111.0 3 3
nan 143.0 3 3
104.0 103.0 3 3
78.0 93.0 3 3
194.0 115.0 3 3
448.0 102.0 3 3
nan 159.0 3 3
nan 181.0 3 3
nan 137.0 3 3
nan 165.0 3 3
159.0 160.0 3 3
nan 199.0 3 3
173.0 178.0 3 3
229.0 223.0 3 3
122.0 71.0 3 3
173.0 82.0 3 3
50.0 49.0 2 2
56.0 55.0 2 2
53.0 47.0 2 2
23.0 15.0 0 0
50.0 46.0 2 2
19.0 17.0 0 0
54.0 52.0 2 2
25.0 22.0 0 0
62.0 61.0 3 3
84.0 74.0 3 3
204.0 199.0 3 3
42.0 41.0 1 1
54.0 50.0 2 2
43.0 32.0 1 1
28.0 27.0 0 0
38.0 37.0 1 1
94.0 69.0 3 3
398.0 64.0 3 3
425.0 73.0 3 3
366.0 77.0 3 3
258.0 112.0 3 3
405.0 77.0 3 3
167.0 90.0 3 3
403.0 102.0 3 3
237.0 105.0 3 3
406.0 97.0 3 3
41.0 34.0 1 1
89.0 88.0 3 3
13.0 12.0 0 0
17.0 15.0 0 0
20.0 19.0 0 0
22.0 21.0 0 0
25.0 24.0 0 0
44.0 39.0 1 1
9.0 8.0 0 0
180.0 68.0 3 3
259.0 64.0 3 3
103.0 81.0 3 3
183.0 91.0 3 3
237.0 110.0 3 3
208.0 85.0 3 3
241.0 87.0 3 3
44.0 43.0 1 1
55.0 45.0 2

In [43]:
hex_data['total_potential_impact'] = hex_data['health_centers_potential_impact'] + hex_data['health_posts_potential_impact'] + hex_data['major_hospitals_potential_impact'] + hex_data['primary_schools_potential_impact'] + hex_data['secondary_schools_potential_impact'] + hex_data['semi_dense_urban_potential_impact']
hex_data['total_current_impact'] = hex_data['health_centers_current_impact'] + hex_data['health_posts_current_impact'] + hex_data['major_hospitals_current_impact'] + hex_data['primary_schools_current_impact'] + hex_data['secondary_schools_current_impact'] + hex_data['semi_dense_urban_current_impact']
hex_data['total_school_current_impact'] =  hex_data['primary_schools_current_impact'] + hex_data['secondary_schools_current_impact']
hex_data['total_school_potential_impact'] =  hex_data['primary_schools_potential_impact'] + hex_data['secondary_schools_potential_impact']
hex_data['total_health_current_impact'] = hex_data['health_centers_current_impact'] + hex_data['health_posts_current_impact'] + hex_data['major_hospitals_current_impact']
hex_data['total_health_potential_impact'] = hex_data['health_centers_potential_impact'] + hex_data['health_posts_potential_impact'] + hex_data['major_hospitals_potential_impact']

In [46]:
def geojson_no_nans(df, path):
    # First write a geopandas dataframe to a geojson file
    df.to_file(path, driver='GeoJSON')

    # Load the GeoJSON data from a file
    with open(path, 'r') as geojson_file:
        geojson_data = json.load(geojson_file)

    def remove_null_properties(feature):
        non_null_properties = {key: value for key, value in feature['properties'].items() if value is not None}
        feature['properties'] = non_null_properties

    # Loop through features and remove null properties
    for feature in geojson_data['features']:
        remove_null_properties(feature)

    # Save the modified GeoJSON to a new file
    with open(path, 'w') as output_file:
        json.dump(geojson_data, output_file, indent=2)

# trimming down data file size

In [47]:
geojson_no_nans(hex_data, './unsynced-data/rwanda/rwa_travel_time_hex-8-staging.geojson')

In [8]:
large_file = gpd.read_file('https://fikamap-web-app.s3.us-west-1.amazonaws.com/data/rwa_travel_time_hex-8.geojson')

In [4]:
pd.set_option('display.max_columns', None)

In [6]:

large_file = large_file.drop("row_col", axis=1)

In [7]:
# write large_file to geojson
large_file.to_file('./unsynced-data/rwanda/rwa_travel_time_hex-8-small.geojson', driver='GeoJSON')

In [11]:
large_file

,h3-index,row_col,female_educational_attainment_mean,females_0_4,females_0_9,females_10_14,females_15_49,females_50_64,females_5_9,females_65_plus,male_educational_attainment_mean,males_0_4,males_0_9,males_10_14,males_15_49,males_50_64,males_5_9,males_65_plus,pop_0_4,pop_0_9,pop_10_14,pop_15_49,pop_50_64,pop_5_9,pop_65_plus,population,pregnancies,births,rwi,underweight,time_delta_constructed_sites_all_education_facilities_fixed,time_delta_constructed_sites_all_health_facilities_optimal,time_delta_constructed_sites_health_centers_optimal,time_delta_constructed_sites_health_posts_optimal,time_delta_constructed_sites_major_hospitals_optimal,time_delta_constructed_sites_primary_schools_fixed,time_delta_constructed_sites_secondary_schools_fixed,time_delta_constructed_sites_semi_dense_urban_optimal,time_delta_no_sites_all_education_facilities_fixed,time_delta_no_sites_all_health_facilities_optimal,time_delta_no_sites_health_centers_optimal,time_delta_no_sites_health_posts_optimal,time_delta_no_sites_major_hospitals_optimal,time_delta_no_sites_primary_schools_fixed,time_delta_no_sites_secondary_schools_fixed,time_delta_no_sites_semi_dense_urban_optimal,travel_time_all_education_facilities_fixed,travel_time_all_health_facilities_optimal,travel_time_constructed_sites_all_education_facilities_fixed,travel_time_constructed_sites_all_health_facilities_optimal,travel_time_constructed_sites_health_centers_optimal,travel_time_constructed_sites_health_posts_optimal,travel_time_constructed_sites_major_hospitals_optimal,travel_time_constructed_sites_primary_schools_fixed,travel_time_constructed_sites_secondary_schools_fixed,travel_time_constructed_sites_semi_dense_urban_optimal,travel_time_health_centers_optimal,travel_time_health_posts_optimal,travel_time_major_hospitals_optimal,travel_time_no_sites_all_education_facilities_fixed,travel_time_no_sites_all_health_facilities_optimal,travel_time_no_sites_health_centers_optimal,travel_time_no_sites_health_posts_optimal,travel_time_no_sites_major_hospitals_optimal,travel_time_no_sites_primary_schools_fixed,travel_time_no_sites_secondary_schools_fixed,travel_time_no_sites_semi_dense_urban_optimal,travel_time_primary_schools_fixed,travel_time_secondary_schools_fixed,travel_time_semi_dense_urban_optimal,all_education_potential_impact,all_education_current_impact,all_health_potential_impact,all_health_current_impact,health_centers_potential_impact,health_centers_current_impact,health_posts_potential_impact,health_posts_current_impact,major_hospitals_potential_impact,major_hospitals_current_impact,primary_schools_potential_impact,primary_schools_current_impact,secondary_schools_potential_impact,secondary_schools_current_impact,semi_dense_urban_potential_impact,semi_dense_urban_current_impact,total_potential_impact,total_current_impact,total_school_current_impact,total_school_potential_impact,total_health_current_impact,total_health_potential_impact,geometry
0,886ad80001fffff,1334_1576&1335_1573&1335_1574&1335_1575&1335_1...,4.6503,17,36,18,71,12,19,4,4.9094,16,36,19,74,12,19,3,33,72,37,146,24,38,8,290,20,14,0.0152,0.0814,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0,67,112,67,112,199,112,332,67.240000,162.600000,185,199,112,332,67,112,199,112,332,67.240000,162.600000,185,67.240000,162.600000,185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((30.17441 -2.16518, 30.17633 -2.16048..."
1,886ad80003fffff,1338_1586&1338_1587&1338_1588&1339_1583&1339_1...,4.5309,25,56,29,116,19,30,7,4.8748,26,57,30,111,16,31,5,52,114,59,228,35,62,12,451,34,24,-0.0153,0.0877,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0,60,104,60,104,180,104,341,60.731707,130.878049,181,180,104,341,60,104,180,104,341,60.731707,130.878049,181,60.731707,130.878049,181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((30.18272 -2.16847, 30.18464 -2.16378..."
2,886ad80005fffff,1343_1564&1344_1563&1344_1564&1344_1572&1344_1...,4.4477,12,29,17,64,10,17,3,4.8480,13

In [13]:
# row_col = just the hex_id and h3-index columns from large_file
row_col = large_file[['h3-index', 'row_col']]

In [15]:
# write dataframe to json
row_col.to_json('./unsynced-data/rwanda/rwa_travel_time_hex-8-row_col.json', orient='records')